# Case: Emneord

**TODO**
- text

Et andet spørgsmål er om vi kan bruge genrerummet til at forbedre metadata. En hypotese er emneord ligge i klumper i genrerummet. I så fald kan vi bruge dette til at have et mål for hvor typisk et materiale er for et emne.

Lad os først kigge på hvorledes metadata ser ud:

In [1]:
import bibdata
from collections import Counter
import numpy
bibdata.meta[1234]

{'_id': '870970-basis:51363035',
 'creator': 'Jakob Martin Strid',
 'idx': 1234,
 'language': 'Dansk',
 'subject-term': ['årstider',
  'for 5 år',
  'sk',
  'sjove bøger',
  'Skønlitteratur',
  'for 4 år',
  'for 6 år',
  'for 3 år',
  'venner',
  'vejret',
  'dyrefortællinger'],
 'title': 'Mimbo Jimbo og den lange vinter',
 'type': 'book'}

Som vi kan se er der, for hvert materiale et antal emneord `subject-term`.

Lad os fokusere på de mest populære emneord. Vi kan finde disse ved at lave en liste af alle anvendte emneord `subjects`, og derefter finde de mest hyppige af disse. Listen af alle emneord finder vi ved at gennemløbe alle bibliografiske poster `bib` og for hver af disse tilføje samtlige emneord `subject` til listen af alle emneord `subjects`. Herefter kan den indbyggede `Counter` i Python finde de hyppigst brugte emneord:

In [2]:
subjects = []
for bib in bibdata.meta:
    for subject in bib.get('subject-term', []):
        subjects.append(subject)
Counter(subjects).most_common(200)

[('sk', 7036),
 ('Skønlitteratur', 6445),
 ('for 6 år', 1109),
 ('krimi', 1100),
 ('for 5 år', 1097),
 ('for 7 år', 1063),
 ('let at læse', 1047),
 ('for 4 år', 999),
 ('for 8 år', 932),
 ('Spillefilm', 904),
 ('Danmark', 867),
 ('kærlighed', 817),
 ('÷', 794),
 ('for 10 år', 791),
 ('for 9 år', 791),
 ('for 3 år', 769),
 ('2000-2009', 746),
 ('77.7', 721),
 ('for begynderlæsere', 673),
 ('for 11 år', 662),
 ('USA', 648),
 ('for børn', 632),
 ('for 12 år', 615),
 ('familien', 553),
 ('spænding', 548),
 ('romaner', 525),
 ('kvinder', 507),
 ('spændende bøger', 473),
 ('England', 456),
 ('Sverige', 429),
 ('Dansk', 423),
 ('2010-2019', 398),
 ('opskrifter', 377),
 ('venskab', 371),
 ('for 13 år', 369),
 ('for 2 år', 357),
 ('historie', 348),
 ('Biografier af enkelte personer', 335),
 ('humor', 330),
 ('Børnefilm', 326),
 ('77.74', 325),
 ('sjove bøger', 324),
 ('for folkeskolen', 312),
 ('for 14 år', 309),
 ('fantasy', 305),
 ('politiromaner', 304),
 ('erindringer', 296),
 ('1990-1999', 

Gad vide hvad `'other'` dækker over:

In [3]:
[bib['title'] for bib in bibdata.meta if bib.get('type') == 'other'][:5]

['Alt for damerne', 'Illustreret videnskab', 'Femina', 'Bo bedre', 'I form']

Vi er i stand til at finde bøgerne med et givent emneord, ved at gennemløbe alle bøgerne (`book_id`), og kun beholde dem der har det pågældende emneord. Hvis vi eksempelvis vil finde børnefilmene kan vi gøre det således:

In [4]:
films = [book_id for book_id in range(len(bibdata.meta))
         if 'Børnefilm' in bibdata.meta[book_id].get('subject-term', [])]

I koden herover er `len(bibdata.meta)` antallet af materialer, og `range(n)` returnere en liste med `n` tal, i.e. `[0,1,2,...]`. Når vi bruger `.get('subject-term', [])` får vi enten en liste af emneord, eller en tom liste hvis metadata mangler, - ellers ville koden fejle hvis vi manglede metadata.

Nu da vi har listen af børnefilm kan vi finde punkterne i genrerummet for disse:

Vi kan herefter udskrive de først 10 film:

In [5]:
print([bibdata.title_creator(film) for film in films[:10]])

['Bamses billedbog dvd - Katrine Hauch-Fausbøll (movie)', 'Kaj og Andrea - Fausbøll Katrine Hauch- (movie)', 'Ninjago - masters of spinjitzu - Sørensen Thomas inv (movie)', 'Pippi - Hellbom Olle (movie)', 'Filmhits for børn -  (movie)', 'Legends of Chima - Andreasen Tommy inv (movie)', 'Op. Med bonus features disc - Docter Pete (movie)', 'Far til fire - på japansk - Glud Thomas (movie)', 'Ratatouille - Walt Disney firma (movie)', 'Alice i Eventyrland - Walt Disney firma (movie)']


Hver af disse film svare til et punkt(vektor) i genrerummet, som vi kan finde således:

In [6]:
vectors = [bibdata.genres[film] for film in films]

Vi forestiller os at materialerne med samme emneord, ligger i en klump i nærheden af hinanden i genrerummet. I så fald er midten af klumpen gennemsnits-punktet(`mean`), og størrelsen af klumpen den statistiske spredning(`std` = standard deviation), hvilket let findes i Python:

In [7]:
mean = numpy.mean(vectors, axis=0)
std = numpy.std(vectors, axis=0)

Vi kan herefter finde ud af hvor tæt et materiale ligger på klumpen, ved at tage afstanden i genrerummet, vægtet med klumpens størrelse (spredning). I.e. vi tager punktet for materialet(`bibdata.genres[book_id]`), trækker gennemsnitspunktet fra klumpen fra(`mean`), dividere med størrelsen(`std`) og tager længden af dette `numpy.linalg.norm(...)`. Så for alice i eventyrland, cirkeline(616) og turen går til berlin(149) er deres børnefilms-agtighed (jo lavere jo bedre):

In [8]:
print('alice i eventyrland(film)', numpy.linalg.norm((bibdata.genres[810] - mean) / std))
print('cirkeline', numpy.linalg.norm((bibdata.genres[616] - mean) / std))
print('berlin', numpy.linalg.norm((bibdata.genres[149] - mean) / std))

alice i eventyrland(film) 11.79158192
cirkeline 19.9179050997
berlin 38.8215720144


Vi kan nu lave en liste af biblioteksmaterialer, kombineret med hvor meget de minder om børnefilm. Hvis vi derefter sorterer disse, finder vi de mest børnefilms-agtige materialer:

In [9]:
weighted_books = [(numpy.linalg.norm((bibdata.genres[book_id]-mean)/std), book_id)
       for book_id in range(len(bibdata.meta))]
weighted_books.sort()
print(weighted_books[:10])

[(6.4345398281629294, 3801), (6.6494780896369434, 738), (6.6877823144785085, 3216), (6.7020911867051352, 5598), (6.7608479969260493, 2992), (6.7826793078348446, 3866), (6.8973703018034787, 1742), (6.9213674230626134, 3148), (6.9241092655234002, 2211), (6.9706191223710414, 2931)]


&dash; hvilket er følgende materialer:

In [10]:
[bibdata.title_creator(book_id) for (weight, book_id) in weighted_books][0:10]

['Shrek - Elliott Ted aus (movie)',
 'Ratatouille - Walt Disney firma (movie)',
 'Ice age 2 : på tynd is - Carlos Saldanha (movie)',
 'Lilo & Stitch - Walt Disney firma (movie)',
 'Find Nemo - Reynolds David (movie)',
 'Shrek 2 - Vernon Conrad drt (movie)',
 'Toy story 2 - Walt Disney firma (movie)',
 'Junglebogen - Walt Disney firma (movie)',
 'Happy feet - Miller George f. 1945 (movie)',
 'Madagascar - Darnell Eric (movie)']

Nu har vi sorteret biblioteksmaterialerne efter børnefilms-agtighed, - dette kunne være praktisk at kunne, for hvilket som helst emneord. Derfor definere vi en ny funktion, som samler denne funktionalitet:

In [11]:
def books_by_subject(subject):
    books = [book_id for book_id in range(len(bibdata.meta)) if subject in bibdata.meta[book_id].get('subject-term', [])]
    vectors = [bibdata.genres[book] for book in books]
    mean = numpy.mean(vectors, axis=0)
    std = numpy.std(vectors, axis=0)
    weighted_books = [(numpy.linalg.norm((bibdata.genres[book_id]-mean)/std), book_id) for book_id in range(10000)]
    sorted_books = [book_id for (weight, book_id) in sorted(weighted_books)]
    return sorted_books

In [12]:
"; ".join([subject for (subject, count) in Counter(subjects).most_common(150)])

'sk; Skønlitteratur; for 6 år; krimi; for 5 år; for 7 år; let at læse; for 4 år; for 8 år; Spillefilm; Danmark; kærlighed; ÷; for 10 år; for 9 år; for 3 år; 2000-2009; 77.7; for begynderlæsere; for 11 år; USA; for børn; for 12 år; familien; spænding; romaner; kvinder; spændende bøger; England; Sverige; Dansk; 2010-2019; opskrifter; venskab; for 13 år; for 2 år; historie; Biografier af enkelte personer; humor; Børnefilm; 77.74; sjove bøger; for folkeskolen; for 14 år; fantasy; politiromaner; erindringer; 1990-1999; rejseførere; piger; Computerspil; 79.41; 1940-1949; computerspil; drama; børn; Frilæs/0.-2. kl.; Norge; 1980-1989; 1970-1979; for 15 år; København; drenge; Billedbøger sk; Folkeskolen; Del af serie; 1960-1969; bk; unge; Læs let; amerikanske film; 1900-1999; parforhold; eventyr; veninder; Fantasy; ægteskab; dameromaner; 1950-1959; Frankrig; seriemordere; Tyskland; gå på opdagelse; Oplæsning 0.- 2. kl.; den 2. verdenskrig; far-søn forholdet; strikning; 1930-1939; forelskelse; i

In [13]:
subject = 'Biografier af enkelte personer'
subject = 'computerspil'
subject = 'sex'
print('\n'.join([bibdata.title_creator(book_id) for book_id in books_by_subject(subject) 
 if not subject in bibdata.meta[book_id].get('subject-term', [])][:5]))
print('---- subject:' + subject + ' ----')
print('\n'.join([bibdata.title_creator(book_id) for book_id in reversed(books_by_subject(subject))
 if subject in bibdata.meta[book_id].get('subject-term', [])][:5]))

Blottet for dig - Sylvia Day (book)
Spejlet i dig - Sylvia Day (book)
Bundet til dig - Sylvia Day (book)
Stolthed og lyst - Sylvia Day (book)
Syv års længsel - Sylvia Day (book)
---- subject:sex ----
Totte og Malene - Gunilla Wolde (book)
Bollevenner - Gluck Will (movie)
Emma 12 år - snart 13 - Kirsten Ahlburg (book)
Nymphomaniac - Claro Manuel cng (movie)
Linas aftenbog - Emma Hamberg (book)


In [14]:
bibdata.meta[9953]

{'_id': '870970-basis:50673103',
 'creator': 'Dowsett Elizabeth edt',
 'idx': 9953,
 'language': 'Dansk',
 'subject-term': ['!',
  'for 12 år',
  'for 11 år',
  'Biografier af enkelte personer',
  'for 9 år',
  'Dave Filoni',
  'Star wars - the clone wars',
  '99.4 Filoni, Dave',
  'Legetøj',
  '79.31',
  'figurer',
  'Dave',
  'for 10 år',
  'Filoni, Dave',
  'Filoni'],
 'title': 'Star wars - clone wars figurleksikon',
 'type': 'book'}

In [15]:
subject = 'let at læse'
# books = [book_id for book_id in range(len(bibdata.meta)) if subject in bibdata.meta[book_id]['subject-term']]
books = [book_id for book_id in range(len(bibdata.meta)) if subject in bibdata.meta[book_id].get('subject-term', [])]
vectors = [bibdata.genres[book] for book in books]
mean = numpy.mean(vectors, axis=0)
std = numpy.std(vectors, axis=0)

subject_books = sorted(
    [(numpy.linalg.norm((bibdata.genres[book_id]-mean)/std, ord=1), book_id) for book_id in range(10000)])
[(bibdata.title_creator(book_id), ",".join(bibdata.meta[book_id].get('subject-term',[]))) for (weight, book_id) in subject_books if not book_id in books][:10]


[('Jorden - Leonie Pratt (book)',
  'Anbefales: Basis,for 9 år,Planeter. Måner,for 8 år,÷,for 7 år,Jorden,52.43,Lette fagbøger'),
 ('Rumfart - Troels Gollander (book)',
  'Rummet,Første Fakta,Sagprosa sm,for 9 år,Faglig læsning,Tværfagligt materiale,for 8 år,for folkeskolen,62.98,Faglig letlæsning,for 7 år,Lette fagbøger,Astronautik,rumfart'),
 ('Kom frem, Bella! - Otto Dickmeiss (book)',
  'for 6-7 år,bange,sk,Skønlitteratur,for 7 år,for 6 år,hunde'),
 ('Aladdin og den magiske lampe - Katie Daynes (book)',
  'for børn,Iran,39.12,eventyr,for 9 år,iranske eventyr,sk,for 8 år,Skønlitteratur,romaner,÷,for 10 år,persiske eventyr,Eventyr, historier og fabler,Anbefales: 2. klasse'),
 ('Alle elsker Sigge - Benedikte Biørn (audiobook)',
  'heste,for 9 år,veninder,sk,for 8 år,Skønlitteratur,for 10 år,piger'),
 ('Pony & co. - Kirsten Sonne Harild (audiobook)',
  'for 11 år,heste,for 9 år,veninder,sk,Skønlitteratur,for 10 år,piger'),
 ('Gode venner og et straffespark - Christian Bieniek (book)',


## Øvelser

- Find listen af de 100 eller 200 mest populære emneord, i stedet for blot de 10 mest populære.
- Find ud af hvilke "`sex`"ede bøger, "`børnefilm`", "`Biografier af enkelte personer`", etc. som er outliers, enten fordi de ligner dette emne og ikke har emneorder, - eller har emneordet og ikke ligner de andre indenfor emnet. Kunne det give mening at fjerne/tilføje emneordet for disse?